In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

In [3]:
# k
ks = [5, 10, 50]

recall = evaluators.RecallEvaluator(test_set, ks)
unpopularity = evaluators.UnpopularityEvaluator(test_set, ks)
f1_score = evaluators.F1ScoreEvaluator(test_set, ks)
coverage = evaluators.CoverageEvaluator(test_set, ks)
hubness = evaluators.HubnessEvaluator(test_set, ks)

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 10
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = False

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [21]:
trainer.fit(n_batch, n_epoch, knn, recall, valid_per_epoch)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:04<00:00, 211.05it/s]


In [22]:
trainer.valid_scores

,Recall@5,Recall@10,Recall@50,epoch,losses
0,0.374698,0.543694,0.876263,0,NaN
0,0.385802,0.557179,0.882815,10,0.244505


In [16]:
trainer.valid(mp, recall)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:12<00:00, 76.29it/s]


In [17]:
trainer.valid_scores

,Recall@5,Recall@10,Recall@50
0,0.294468,0.451634,0.834788


In [18]:
mp2 = searches.MutualProximity(model, gaussian)

In [19]:
trainer.valid(mp, recall)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:12<00:00, 75.30it/s]


In [20]:
trainer.valid_scores

,Recall@5,Recall@10,Recall@50
0,0.291078,0.454359,0.836245
